# Midterm Project - Apriori Algorithm

**Author** : *Joshua Kobuskie*

## Part 1 Details
1. Create 10 (or any number of, not less than 5) items usually seen in
Amazon, K-mart, or any other supermarkets (e.g. diapers, clothes, etc.).
2. Create a database of at least 20 transactions each containing some of these
items. Save the transaction in a CSV file.
3. Repeat (1) by creating 4 additional, different databases each containing at
least 20 transactions.
4. Note: You can create these transactions and datasets manually, download
them from the net, or use the examples I will provide. In any case, add a
note to your report where and how you built your data sets.
5. The items and transaction must not be random so that your code is
deterministic

**NOTE: May need to add more data to generic and custom transactions**

In [174]:
# Create Amazon transactions from provided example
import csv

amazonData  = [
["A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch"],
["A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies"],
["A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition"],
["Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition", "Beginning Programming with Java"],
["Android Programming: The Big Nerd Ranch", "Beginning Programming with Java", "Java 8 Pocket Guide"],
["A Beginner’s Guide", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition"],
["A Beginner’s Guide", "Head First Java 2nd Edition", "Beginning Programming with Java"],
["Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch"],
["Java For Dummies", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition", "Beginning Programming with Java"],
["Beginning Programming with Java", "Java 8 Pocket Guide", "C++ Programming in Easy Steps"],
["A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch"],
["A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "HTML and CSS: Design and Build Websites"],
["A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Java 8 Pocket Guide", "HTML and CSS: Design and Build Websites"],
["Java For Dummies", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition"],
["Java For Dummies", "Android Programming: The Big Nerd Ranch"],
["A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch"],
["A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch"],
["Head First Java 2nd Edition", "Beginning Programming with Java", "Java 8 Pocket Guide"],
["Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition"],
["A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies"]
]

with open("amazonTransactions.csv", "w") as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(amazonData)

# amazonSet = ["A Beginner’s Guide", "Java: The Complete Reference", "Java For Dummies", "Android Programming: The Big Nerd Ranch", "Head First Java 2nd Edition", "Beginning Programming with Java", "Java 8 Pocket Guide", "C++ Programming in Easy Steps", "Effective Java (2nd Edition)", "HTML and CSS: Design and Build Websites"]

# with open("amazonSets.csv", "w") as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerow(amazonSet)

In [175]:
# Create Best Buy transactions from provided example:

bestBuyData = [
["Desk Top", "Printer", "Flash Drive", "Microsoft Office", "Speakers", "Anti-Virus"],
["Lab Top", "Flash Drive", "Microsoft Office", "Lab Top Case", "Anti-Virus"],
["Lab Top", "Printer", "Flash Drive", "Microsoft Office", "Anti-Virus", "Lab Top Case", "External Hard-Drive"],
["Lab Top", "Printer", "Flash Drive", "Anti-Virus", "External Hard-Drive", "Lab Top Case"],
["Lab Top", "Flash Drive", "Lab Top Case", "Anti-Virus"],
["Lab Top", "Printer", "Flash Drive", "Microsoft Office"],
["Desk Top", "Printer", "Flash Drive", "Microsoft Office"],
["Lab Top", "External Hard-Drive", "Anti-Virus"],
["Desk Top", "Printer", "Flash Drive", "Microsoft Office", "Lab Top Case", "Anti-Virus", "Speakers", "External Hard-Drive"],
["Digital Camera", "Lab Top", "Desk Top", "Printer", "Flash Drive", "Microsoft Office", "Lab Top Case", "Anti-Virus", "External Hard-Drive", "Speakers"],
["Lab Top", "Desk Top", "Lab Top Case", "External Hard-Drive", "Speakers", "Anti-Virus"],
["Digital Camera", "Lab Top", "Lab Top Case", "External Hard-Drive", "Anti-Virus", "Speakers"],
["Digital Camera", "Speakers"],
["Digital Camera", "Desk Top", "Printer", "Flash Drive", "Microsoft Office"],
["Printer", "Flash Drive", "Microsoft Office", "Anti-Virus", "Lab Top Case", "Speakers", "External Hard-Drive"],
["Digital Camera", "Flash Drive", "Microsoft Office", "Anti-Virus", "Lab Top Case", "External Hard-Drive", "Speakers"],
["Digital Camera", "Lab Top", "Lab Top Case"],
["Digital Camera", "Lab Top Case", "Speakers"],
["Digital Camera", "Lab Top", "Printer", "Flash Drive", "Microsoft Office", "Speakers", "Lab Top Case", "Anti-Virus"],
["Digital Camera", "Lab Top", "Speakers", "Anti-Virus", "Lab Top Case"]
]

with open("bestBuyTransactions.csv", "w") as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(bestBuyData)

# bestBuySet = ["Digital Camera", "Lab Top", "Desk Top", "Printer", "Flash Drive", "Microsoft Office", "Speakers", "Lab Top Case", "Anti-Virus", "External Hard-Drive"]

# with open("bestBuySets.csv", "w") as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerow(bestBuySet)

In [176]:
# Create K-Mart transactions from provided example:
kMartData = [
["Decorative Pillows", "Quilts", "Embroidered Bedspread"],
["Embroidered Bedspread", "Shams", "Kids Bedding", "Bedding Collections", "Bed Skirts", "Bedspreads", "Sheets"],
["Decorative Pillows", "Quilts", "Embroidered Bedspread", "Shams", "Kids Bedding", "Bedding Collections"],
["Kids Bedding", "Bedding Collections", "Sheets", "Bedspreads", "Bed Skirts"],
["Decorative Pillows", "Kids Bedding", "Bedding Collections", "Sheets", "Bed Skirts", "Bedspreads"],
["Bedding Collections", "Bedspreads", "Bed Skirts", "Sheets", "Shams", "Kids Bedding"],
["Decorative Pillows", "Quilts"],
["Decorative Pillows", "Quilts", "Embroidered Bedspread"],
["Bedspreads", "Bed Skirts", "Shams", "Kids Bedding", "Sheets"],
["Quilts", "Embroidered Bedspread", "Bedding Collections"],
["Bedding Collections", "Bedspreads", "Bed Skirts", "Kids Bedding", "Shams", "Sheets"],
["Decorative Pillows", "Quilts"],
["Embroidered Bedspread", "Shams"],
["Sheets", "Shams", "Bed Skirts", "Kids Bedding"],
["Decorative Pillows", "Quilts"],
["Decorative Pillows", "Kids Bedding", "Bed Skirts", "Shams"],
["Decorative Pillows", "Shams", "Bed Skirts"],
["Quilts", "Sheets", "Kids Bedding"],
["Shams", "Bed Skirts", "Kids Bedding", "Sheets"],
["Decorative Pillows", "Bedspreads", "Shams", "Sheets", "Bed Skirts", "Kids Bedding"]
]

with open("kMartTransactions.csv", "w") as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(kMartData)

# kMartSet = ["Quilts", "Bedspreads", "Decorative Pillows", "Bed Skirts", "Sheets", "Shams", "Bedding Collections", "Kids Bedding", "Embroidered Bedspread", "Towels"]
            
# with open("kMartSets.csv", "w") as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerow(kMartSet)

In [177]:
# Create Nike transactions from provided example:
# Corrected Dry Fir V-Nick to Dry Fit V-Neck
# Corrected one line with Dry to Dry Fit V-Neck

nikeData = [
["Running Shoe", "Socks", "Sweatshirts", "Modern Pants"],
["Running Shoe", "Socks", "Sweatshirts"],
["Running Shoe", "Socks", "Sweatshirts", "Modern Pants"],
["Running Shoe", "Sweatshirts", "Modern Pants"],
["Running Shoe", "Socks", "Sweatshirts", "Modern Pants", "Soccer Shoe"],
["Running Shoe", "Socks", "Sweatshirts"],
["Running Shoe", "Socks", "Sweatshirts", "Modern Pants", "Tech Pants", "Rash Guard", "Hoodies"],
["Swimming Shirt", "Socks", "Sweatshirts"],
["Swimming Shirt", "Rash Guard", "Dry Fit V-Neck", "Hoodies", "Tech Pants"],
["Swimming Shirt", "Rash Guard", "Dry Fit V-Neck"],
["Swimming Shirt", "Rash Guard", "Dry Fit V-Neck"],
["Running Shoe", "Swimming Shirt", "Socks", "Sweatshirts", "Modern Pants", "Soccer Shoe", "Rash Guard", "Hoodies", "Tech Pants", "Dry Fit V-Neck"],
["Running Shoe", "Swimming Shirt", "Socks", "Sweatshirts", "Modern Pants", "Soccer Shoe", "Rash Guard", "Tech Pants", "Dry Fit V-Neck", "Hoodies"],
["Running Shoe", "Swimming Shirt", "Rash Guard", "Tech Pants", "Hoodies", "Dry Fit V-Neck"],
["Running Shoe", "Swimming Shirt", "Socks", "Sweatshirts", "Modern Pants", "Dry Fit V-Neck", "Rash Guard", "Tech Pants"],
["Swimming Shirt", "Soccer Shoe", "Hoodies", "Dry Fit V-Neck", "Tech Pants", "Rash Guard"],
["Running Shoe", "Socks"],
["Socks", "Sweatshirts", "Modern Pants", "Soccer Shoe", "Hoodies", "Rash Guard", "Tech Pants", "Dry Fit V-Neck"],
["Running Shoe", "Swimming Shirt", "Rash Guard"],
["Running Shoe", "Swimming Shirt", "Socks", "Sweatshirts", "Modern Pants", "Soccer Shoe", "Hoodies", "Tech Pants", "Rash Guard", "Dry Fit V-Neck"]
]

with open("nikeTransactions.csv", "w") as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(nikeData)

# nikeSet = ["Running Shoe", "Soccer Shoe", "Socks", "Swimming Shirt", "Dry Fit V-Neck", "Rash Guard", "Sweatshirts", "Hoodies", "Tech Pants", "Modern Pants"]

# with open("nikeSets.csv", "w") as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerow(nikeSet)

In [178]:
# Create Generic transactions from provided example:
genericData = [
["A", "B", "C"],
["A", "B", "C"],
["A", "B", "C", "D"],
["A", "B", "C", "D", "E"],
["A", "B", "D", "E"],
["A", "D", "E"],
["A", "E"],
["A", "E"],
["A", "C", "E"],
["A", "C", "E"],
["A", "C", "E"]
]

with open("genericTransactions.csv", "w") as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(genericData)

# genericSets = ["A", "B", "C", "D", "E", "F"]

# with open("genericSets.csv", "w") as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerow(genericSets)

In [179]:
# Create Custom transactions from provided example:

customData = [
["ink", "pen", "cheese", "bag"],
["milk", "pen", "juice", "cheese"],
["milk", "juice"],
["juice", "milk", "cheese"],
["ink", "pen", "cheese", "bag"],
["milk", "pen", "juice", "cheese"],
]

with open("customTransactions.csv", "w") as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(customData)

# customSet = ["ink", "pen", "cheese", "bag", "juice", "milk"]

# with open("customSets.csv", "w") as csvFile:
#     writer = csv.writer(csvFile)
#     writer.writerow(customSet)

## Part 2 Details
* Implement the brute force method to generate frequent items and generate
association rules.
* The brute force method for finding frequent itemsets works as follows.
Enumerate and generate all possible 1-itemsets and 2-itemsets. There are
30 items, so there are 435 possible 2-itemsets totally. Check to see whether
each possible 1-itemset/2-itemset is frequent. Then enumerate and generate
all possible 3-itemsets. There are 4060 possible 3-itemsets totally. Check to
see whether each possible 3-itemset is :frequent. Keep on doing so until
you see none of the possible k-itemsets is :frequent for some k, at which
point the brute force method terminates without generating (k+ 1 )-
itemsets.

In [180]:
# The data set selection, support, and confidence must be user-specified parameters

# Start and take in user input

print("Apriori Midterm Project - Joshua Kobuskie")

dataSelect = input("Please select a dataset by number: \n1. Amazon \n2. Best Buy \n3. K-Mart \n4. Nike \n5. Generic \n6. Custom\n")

# Confirm selection is valid
try:
    dataSelect = int(dataSelect)
    if dataSelect not in range(1,7):
        print("Invalid input. Please restart and enter a valid dataset number.")
        raise SystemExit
except ValueError:
    print("Invalid input. Please restart and enter a valid dataset number.")
    raise SystemExit

datasets = ["Amazon", "Best Buy", "K-Mart", "Nike", "Generic", "Custom"]

print("You have selected dataset {}: {}".format(dataSelect, datasets[dataSelect-1]))

support = input("Please select the Minimum Support Level in % (value 1 to 100):\n")

# Confirm selection is valid
try:
    support = int(support)
    if support not in range(1,101):
        print("Invalid input. Please restart and enter a valid support level.")
        raise SystemExit
except ValueError:
    print("Invalid input. Please restart and enter a valid support level.")
    raise SystemExit

print("You have selected a support level of {}%".format(support))

confidence = input("Please select the Minimum Confidence Level in % (value 1 to 100):\n")

# Confirm selection is valid
try:
    confidence = int(confidence)
    if confidence not in range(1,101):
        print("Invalid input. Please restart and enter a valid confidence level.")
        raise SystemExit
except ValueError:
    print("Invalid input. Please restart and enter a valid confidence level.")
    raise SystemExit

print("You have selected a confidence level of {}%".format(confidence))

Apriori Midterm Project - Joshua Kobuskie
You have selected dataset 1: Amazon
You have selected a support level of 50%
You have selected a confidence level of 50%


In [181]:
# Open selected sets and transactions
import itertools

support /= 100
confidence /= 100

transactions = ["amazonTransactions.csv", "bestBuyTransactions.csv", "kMartTransactions.csv", "nikeTransactions.csv", "genericTransactions.csv", "customTransactions.csv"]

with open(transactions[dataSelect], mode ='r') as file:
  csvFile = csv.reader(file)
  curTrans = []
  for line in csvFile:
    curTrans.append(line)

# Having the starting sets is redundant. Removing and just find unique items in transactions

curSet = set()
for trans in curTrans:
  for item in trans:
    # Will only add if the item has not been encountered yet since it is a set
    curSet.add(item)

frequentItems = {}

# Check all sizes of combinations until one is not found or reach max len
# Originally was going to limit the size of the combinations as being checked, but upon review realized that this was not truly brute force
# Removed optimization

found = 1
i = 1
while (i < len(curSet)+1 and found > 0):
  found = 0

  # For each combination of size i
  for combination in itertools.combinations(curSet, i):
    # Count support of combination by iterating through the transactions
    count = 0

    for trans in curTrans:
      # Cast combination to a set to use the issubset function
      # Check if combination occurs in transaction and incriment count
      if set(combination).issubset(trans):
        count += 1
    
    if count/len(curTrans) >= support:
      # Save frequent items
      frequentItems[combination] = count/len(curTrans)
      # Found at least one frequent item in this size of combinations
      found += 1

  # Check next combination size
  i += 1

print("\nFrequent Items\n")

for i, (items, supp) in enumerate(frequentItems.items()):
    print("Frequent Item {}: [{}]\nSupport: {:0.2f}%\n".format(i+1, ", ".join(items), supp*100))


Frequent Items

Frequent Item 1: [Microsoft Office]
Support: 55.00%

Frequent Item 2: [Printer]
Support: 50.00%

Frequent Item 3: [Lab Top Case]
Support: 70.00%

Frequent Item 4: [Flash Drive]
Support: 65.00%

Frequent Item 5: [Speakers]
Support: 55.00%

Frequent Item 6: [Lab Top]
Support: 60.00%

Frequent Item 7: [Anti-Virus]
Support: 70.00%

Frequent Item 8: [Microsoft Office, Flash Drive]
Support: 55.00%

Frequent Item 9: [Printer, Flash Drive]
Support: 50.00%

Frequent Item 10: [Lab Top Case, Lab Top]
Support: 50.00%

Frequent Item 11: [Lab Top Case, Anti-Virus]
Support: 60.00%

Frequent Item 12: [Flash Drive, Anti-Virus]
Support: 50.00%

Frequent Item 13: [Lab Top, Anti-Virus]
Support: 50.00%



In [182]:
associationRules = []

for itemset in frequentItems:
    if len(itemset) > 1:

        # For all subsets of the itemset
        for i in range(1, len(itemset)):
            for ant in itertools.combinations(itemset, i):
                # Check confidence
                if frequentItems[itemset]/frequentItems[ant] >= confidence:
                    cons = tuple(set(itemset) - set(ant))
                    # Store antecedent, consequent, confidence, support
                    associationRules.append((ant, cons, frequentItems[itemset]/frequentItems[ant], frequentItems[itemset]))

print("\nAssociation Rules\n")

for i in range(len(associationRules)):
    print("Rule {}: [{}] ->  [{}]\nConfidence: {:0.2f}%\nSupport: {:0.2f}%\n".format(i+1, ", ".join(associationRules[i][0]), " ".join(associationRules[i][1]), associationRules[i][2]*100, associationRules[i][3]*100))


Association Rules

Rule 1: [Microsoft Office] ->  [Flash Drive]
Confidence: 100.00%
Support: 55.00%

Rule 2: [Flash Drive] ->  [Microsoft Office]
Confidence: 84.62%
Support: 55.00%

Rule 3: [Printer] ->  [Flash Drive]
Confidence: 100.00%
Support: 50.00%

Rule 4: [Flash Drive] ->  [Printer]
Confidence: 76.92%
Support: 50.00%

Rule 5: [Lab Top Case] ->  [Lab Top]
Confidence: 71.43%
Support: 50.00%

Rule 6: [Lab Top] ->  [Lab Top Case]
Confidence: 83.33%
Support: 50.00%

Rule 7: [Lab Top Case] ->  [Anti-Virus]
Confidence: 85.71%
Support: 60.00%

Rule 8: [Anti-Virus] ->  [Lab Top Case]
Confidence: 85.71%
Support: 60.00%

Rule 9: [Flash Drive] ->  [Anti-Virus]
Confidence: 76.92%
Support: 50.00%

Rule 10: [Anti-Virus] ->  [Flash Drive]
Confidence: 71.43%
Support: 50.00%

Rule 11: [Lab Top] ->  [Anti-Virus]
Confidence: 83.33%
Support: 50.00%

Rule 12: [Anti-Virus] ->  [Lab Top]
Confidence: 71.43%
Support: 50.00%



## Part 3 Details
Use an existing Apriori implementation from Python libraries/packages to
verify the results from your brute force algorithm implementation.
* Use Python existing package for fpgrowth (as known as fp-tree algorithm)
to generate the items and rules.
* Compare the results from your brute-force, Apriori, and FP-Tree/Growth.
* Do the three algorithms produce the same results?
* Report the timing performance for all three algorithms as well.
* Which one is faster?
So, for all three algorithms, generate and print out all the association rules and
the input transactions for each of the 5 transactional databases you created/used.
The data set selection, support, and confidence must be user-specified
parameters, so the output should show different rules with respect to different
databases and different support/confidence.
Make sure to show multiple support and confidence results for each data set.
You should prompt the user only once for the input and reuse for the three
algorithms in each run.
The items and transactions must be clear and easy to identify. Your
program should show the performance time for each algorithm.

## Part 4 Details
Github & Jupyter Notebook.
* After you finish your code in development and testing
and make sure it works, and prepare the report (meaning
all heavy lifting job is done ), Create a Github
repository in https://github.com/. Your account must be
with your NJIT email not your personal email (unless if
you have to, but indicate that in your report as well).
* Load your project to the repository.
* Create Jupyter notebook for your work to show the
output, for more info visit https://jupyter.org/
* Give me ya54@njit.edu access as a collaborator to your
repository. (If we have a grader, you give him/her access
too).
* Add Github link to your repository to your report.